In [5]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
"John likes to watch movies, Mary likes movies too",
"John also likes to watch football games",
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray())

['also', 'football', 'games', 'john', 'likes', 'mary', 'movies', 'to', 'too', 'watch']
[[0 0 0 1 2 1 2 1 1 1]
 [1 1 1 1 1 0 0 1 0 1]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [1]:
import gensim
import jieba
import numpy as np
from scipy.linalg import norm
model_file = r"C:\Users\CANDY\Desktop\毕设\草稿\news_12g_baidubaike_20g_novel_90g_embedding_64.bin"
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True)
#调用了一个现成的基于百度百科训练的模型（但是只支持纯文本）
def vector_similarity(s1, s2):
    def sentence_vector(s):
        words = jieba.lcut(s)
        v = np.zeros(64)
        for word in words:
            v += model[word]
        v /= len(words)
        return v
 
    v1, v2 = sentence_vector(s1), sentence_vector(s2)
    return np.dot(v1, v2) / (norm(v1) * norm(v2))

In [2]:
s1 = '生活日常'
s2 = '食品饮料'
vector_similarity(s1, s2)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\CANDY\AppData\Local\Temp\jieba.cache
Loading model cost 0.786 seconds.
Prefix dict has been built successfully.


0.1775401052355638

In [17]:
s1 = '生活搞笑'
s2 = '美妆护理'
vector_similarity(s1, s2)

0.24692325069819782

In [18]:
s1 = '生活日常'
s2 = '美妆护理'
vector_similarity(s1, s2)

0.34455890162607267

In [19]:
s1 = '生活搞笑'
s2 = '食品饮料'
vector_similarity(s1, s2)

0.11199472586549872

In [23]:
s1 = '生活出行'
s2 = '美妆护理'
vector_similarity(s1, s2)

0.26612935093729895

In [21]:
s1 = '生活三农'
s2 = '医药保健'
vector_similarity(s1, s2)

0.17204053281157017

In [22]:
s1 = '生活搞笑'
s2 = '美妆护理'
vector_similarity(s1, s2)

0.24692325069819782

In [13]:
s1 = '生活日常'
s2 = '美妆护理'
vector_similarity(s1, s2)

0.3877587921711751

目前有五种计算图片相似度

1：均值哈希算法（Average hash algorithm）基于比较灰度图每个像素与平均值来实现的，最适用于缩略图，放大图搜索。

优点：
    1：算法简单计算速度快
    2：图像放大，缩小，改变纵横比，或增加，减少亮度，对比度，颜色对hash影响不大

缺点：
    对图片内容敏感，如果内容改变，图片hash改变比较大
    
2：差值哈希算法（Different hash algorithm）
相比aHash，dHash在效率几乎相同的情况下的效果要更好，它是基于渐变实现的

3：感知哈希算法（perceptual hash algorithm）采用的是DCT（离散余弦变换）来降低频率的方法

优点：
    简单快速，准确度更大

缺点：
    图片内容稍微添加几个字或删除几个字影响比较大
    
均值哈希算法、差值哈希算法和感知哈希算法都是值越小，相似度越高，取值为0-64，即汉明距离中，64位的hash值有多少不同。 三直方图和单通道直方图的值为0-1，值越大，相似度越高

In [14]:
import cv2
import numpy as np

# Hash值对比
def cmpHash(hash1, hash2,shape=(10,10)):
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1)!=len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 相等则n计数+1，n最终为相似度
        if hash1[i] == hash2[i]:
            n = n + 1
    return n/(shape[0]*shape[1])
# 感知哈希算法(pHash)
def pHash(img,shape=(10,10)):
    # 缩放32*32
    img = cv2.resize(img, (32, 32))  # , interpolation=cv2.INTER_CUBIC

    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 将灰度图转为浮点型，再进行dct变换
    dct = cv2.dct(np.float32(gray))
    # opencv实现的掩码操作
    dct_roi = dct[0:10, 0:10]

    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash

In [15]:
# 通过得到RGB每个通道的直方图来计算相似度
def classify_hist_with_split(image1, image2, size=(256, 256)):
    # 将图像resize后，分离为RGB三个通道，再计算每个通道的相似值
    image1 = cv2.resize(image1, size)
    image2 = cv2.resize(image2, size)
    sub_image1 = cv2.split(image1)
    sub_image2 = cv2.split(image2)
    sub_data = 0
    for im1, im2 in zip(sub_image1, sub_image2):
        sub_data += calculate(im1, im2)
    sub_data = sub_data / 3
    return sub_data


# 计算单通道的直方图的相似值
def calculate(image1, image2):
    hist1 = cv2.calcHist([image1], [0], None, [256], [0.0, 255.0])
    hist2 = cv2.calcHist([image2], [0], None, [256], [0.0, 255.0])
    # 计算直方图的重合度
    degree = 0
    for i in range(len(hist1)):
        if hist1[i] != hist2[i]:
            degree = degree + (1 - abs(hist1[i] - hist2[i]) / max(hist1[i], hist2[i]))
        else:
            degree = degree + 1
    degree = degree / len(hist1)
    return degree

In [16]:
# 计算单通道的直方图的相似值
def calculate(image1, image2):
    hist1 = cv2.calcHist([image1], [0], None, [256], [0.0, 255.0])
    hist2 = cv2.calcHist([image2], [0], None, [256], [0.0, 255.0])
    # 计算直方图的重合度
    degree = 0
    for i in range(len(hist1)):
        if hist1[i] != hist2[i]:
            degree = degree + (1 - abs(hist1[i] - hist2[i]) / max(hist1[i], hist2[i]))
        else:
            degree = degree + 1
    degree = degree / len(hist1)
    return degree

In [18]:
img1 = cv2.imread(r"C:\Users\CANDY\Desktop\pppp\12.jpg")  
img2 = cv2.imread(r"C:\Users\CANDY\Desktop\pppp\11.jpg")
hash1 = pHash(img1)
hash2 = pHash(img2)
n = cmpHash(hash1, hash2)
print('感知哈希算法相似度：', n)

n = classify_hist_with_split(img1, img2)
print('三直方图算法相似度：', n[0])

n = calculate(img1, img2)
print('单通道的直方图算法相似度：', n[0])

感知哈希算法相似度： 0.54
三直方图算法相似度： 0.7312866
单通道的直方图算法相似度： 0.73664105
